In [1]:
# US Open Pool Scraper

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import csv

In [19]:
url = "https://cuescore.com/tournament/US+Open+9-ball+Championship+2019/5185877"
my_page = requests.get(url)


In [20]:
# Use content 
soup = BeautifulSoup(my_page.content, "html.parser")

In [21]:
# Get div of interest
table = soup.find('table', attrs={'class' : 'score'})


In [22]:
trtags = table.find_all('tr',  attrs={'class' : 'match'})

# Helper Function to get information 
def get_info(winner, winnerScore, loser, loserScore): 
    result = {}
    result["winner"] = winner.find('div', attrs={'class': 'name'}).text.strip()
    result["winnerNationality"] = winner.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["winnerRunouts"] = winner.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["winnerRunouts"] == '':
        result["winnerRunouts"] = 0
    result["winnerScore"] = winnerScore

    result["loser"] = loser.find('div', attrs={'class': 'name'}).text.strip()
    result["loserNationality"] = loser.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["loserRunouts"] = loser.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["loserRunouts"] == '':
        result["loserRunouts"] = 0
    result["loserScore"] = loserScore
    return(result)

results_lst = []
for row in trtags: 
    
    # Determine winner and loser
    playerAScore = row.find('td', attrs={'class': 'scoreA'}).find('input').get("value")
    playerBScore = row.find('td', attrs={'class': 'scoreB'}).find('input').get("value")
    if playerAScore > playerBScore: 
        winner = row.find('td', attrs={'class': 'playerA'})
        loser = row.find('td', attrs={'class': 'playerB'})
        result = get_info(winner, playerAScore, loser, playerBScore)
    else: 
        winner = row.find('td', attrs={'class': 'playerB'})
        loser = row.find('td', attrs={'class': 'playerA'})
        result = get_info(winner, playerBScore, loser, playerAScore)
    
    # Get Match Number
    result['matchNum'] = row.find('td', attrs={'class': 'matchno'}).text.strip()
    # Get Match Time
    
    result['matchTime'] = row.find('td', attrs={'class': 'time'}).get('title')
    
    if result['winnerScore'] == "FF": 
        pass
    else: 
        results_lst.append(result)  

    



In [23]:
usopen = pd.DataFrame(results_lst)

In [24]:
usopen["tournament"] = "US Open 9 Ball Championship 2019"

In [25]:
usopen

,winner,winnerNationality,winnerRunouts,winnerScore,loser,loserNationality,loserRunouts,loserScore,matchNum,matchTime,tournament
0,Jayson Shaw,Scotland,2,11,Marcus Westen,Germany,0,0,1,21.Apr 2019 08:43,US Open 9 Ball Championship 2019
1,Jason Williams,United States,0,4,Tom Staveley,United Kingdom,1,11,2,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
2,Hunter Lombardo,United States,0,11,Kosuke Tojo,Japan,2,10,3,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
3,Chad Rhodes,United States,0,4,Radwan Jameel R Sorouji,Saudi Arabia,0,11,4,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
4,Naoyuki Oi,Japan,0,9,Marco Vignola,Italy,0,11,5,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
...,...,...,...,...,...,...,...,...,...,...,...
490,Chang Yu-Lung,Chinese Taipei,0,11,Liu Haitao,China,0,10,499,25.Apr 2019 18:00,US Open 9 Ball Championship 2019
491,Wu Jiaqing,China,0,11,Wang Can,China,0,0,500,25.Apr 2019 20:57,US Open 9 Ball Championship 2019
492,Jeffrey De Luna,Philippines,2,8,Joshua Filler,Germany,0,11,501,26.Apr 2019 09:00,US Open 9 Ball Championship 2019
493,Chang Yu-Lung,Chinese Taipei,0,5,Wu Jiaqing,China,0,11,502,26.Apr 2019 11:30,US Open 9 Ball Championship 2019


In [26]:
players = pd.concat([usopen.winner, usopen.loser] )

In [27]:
players.value_counts()

Joshua Filler            11
Wu Jiaqing               10
Wang Can                 10
Tyler Styer               9
Marco Teutscher           9
                         ..
Thomas Welle              2
Damian Pongpanki          2
Tyler Fleshman            2
Jamie White               1
Fahad Salem N Alharbi     1
Length: 253, dtype: int64

In [30]:
# How many players
nplayers = len(players.value_counts())
nplayers
usopen.winnerScore = usopen.winnerScore.astype(str).astype(int)
usopen.loserScore = usopen.loserScore.astype(str).astype(int)
usopen.matchNum = usopen.matchNum.astype(str).astype(int)

In [31]:
usopen.dtypes

winner               object
winnerNationality    object
winnerRunouts        object
winnerScore           int64
loser                object
loserNationality     object
loserRunouts         object
loserScore            int64
matchNum              int64
matchTime            object
tournament           object
dtype: object

In [32]:
usopen.to_csv("data/usopen2019.csv")

In [87]:
# Plan 
    # Go to https://wpapool.com/ranking/ 
    # look at all the qualifying events
    # Scrape scores from Cuescore 
    # Get priors on Elo system 
    # Use Priors to predict future tournaments 

    

# Turning Stone Classic XXVII

In [33]:
url = "https://cuescore.com/tournament/Turning+Stone+Classic+XXVII+Mid+Season+Championship/1284289"
my_page = requests.get(url)

In [34]:
# Use content 
soup = BeautifulSoup(my_page.content, "html.parser")

In [35]:
# Get div of interest
table = soup.find('table', attrs={'class' : 'score'})


In [36]:
trtags = table.find_all('tr',  attrs={'class' : 'match'})

# Helper Function to get information 
def get_info(winner, winnerScore, loser, loserScore): 
    result = {}
    result["winner"] = winner.find('div', attrs={'class': 'name'}).text.strip()
    result["winnerNationality"] = winner.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["winnerRunouts"] = winner.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["winnerRunouts"] == '':
        result["winnerRunouts"] = 0
    result["winnerScore"] = winnerScore

    result["loser"] = loser.find('div', attrs={'class': 'name'}).text.strip()
    result["loserNationality"] = loser.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["loserRunouts"] = loser.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["loserRunouts"] == '':
        result["loserRunouts"] = 0
    result["loserScore"] = loserScore
    return(result)

results_lst = []
for row in trtags: 
    
    # Determine winner and loser
    playerAScore = row.find('td', attrs={'class': 'scoreA'}).find('input').get("value")
    playerBScore = row.find('td', attrs={'class': 'scoreB'}).find('input').get("value")
    if playerAScore > playerBScore: 
        winner = row.find('td', attrs={'class': 'playerA'})
        loser = row.find('td', attrs={'class': 'playerB'})
        result = get_info(winner, playerAScore, loser, playerBScore)
    else: 
        winner = row.find('td', attrs={'class': 'playerB'})
        loser = row.find('td', attrs={'class': 'playerA'})
        result = get_info(winner, playerBScore, loser, playerAScore)
        
    # Get Match Number
    result['matchNum'] = row.find('td', attrs={'class': 'matchno'}).text.strip()
    # Get Match Time
    
    result['matchTime'] = row.find('td', attrs={'class': 'time'}).get('title')
    
    if result['winnerScore'] == "FF": 
        pass
    else: 
        results_lst.append(result)  


    



In [37]:
turningstone = pd.DataFrame(results_lst)


In [38]:
players = pd.concat([turningstone.winner, turningstone.loser] )
playerlist = players.value_counts().index
playerlist == "Albin Ouschan"

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [39]:
turningstone.to_csv("data/turningstone2017.csv")

In [108]:
turningstone

,winner,winnerNationality,winnerRunouts,winnerScore,loser,loserNationality,loserRunouts,loserScore,matchNum,matchTime
0,Mike Yednak,United States,0,9,Roger Miller,Canada,0,5,1,06.Jan 2017 18:23
1,Johnny Archer,United States,0,9,Jamie Addessi,United States,0,2,2,06.Jan 2017 18:23
2,Jonathan Smith,United States,0,9,Brian Trinci,United States,0,5,3,06.Jan 2017 18:23
3,Oscar Dominguez,United States,0,9,Mike Giurleo,United States,0,3,4,06.Jan 2017 18:23
4,Chris Braiman,United States,0,9,Robert Ferry,United States,0,7,5,06.Jan 2017 18:23
...,...,...,...,...,...,...,...,...,...,...
249,Dennis Hatch,United States,0,9,John Morra,Canada,0,2,250,08.Jan 2017 13:23
250,Jayson Shaw,Scotland,0,9,Rodney Morris,United States,0,3,251,08.Jan 2017 15:01
251,Dennis Hatch,United States,0,9,Amar Kang,India,0,8,252,08.Jan 2017 15:00
252,Rodney Morris,United States,0,9,Dennis Hatch,United States,0,3,253,08.Jan 2017 17:15


# Treviso 

In [122]:
url = "https://www.epbf.com/tournaments/eurotour/draw-results/"
my_page = requests.get(url)
# Use content 
soup = BeautifulSoup(my_page.content, "html.parser")
# Get div of interest
table = soup.find('table', attrs={'class' : 'table'})
# Then I just did it manually

### WPA Wikipedia 

In [165]:
import regex as re

def wpa_scraper(tables, begin, end):
    results_lst = []
    matchIndex = 1 

    # By inspection, choose interesting tables from beginning to end.
    for index in range(begin, end + 1, 1):
        table = tables[index]

        # Table
        table = table.dropna(axis=1, how='all').dropna(axis=0, how='all')

        new_header = table.iloc[0] #grab the first row for the header
        table = table[1:] #take the data less the header row
        table.drop(table.tail(1).index,inplace=True) # drop last n rows
        table.columns = new_header #set the header row as the df header      
        tablecopy = table.copy()

        table = table.loc[:, table.columns.notnull()]
        nonnancols = list(table.columns.unique())
        for idx, col in enumerate(nonnancols):
            if idx == (len(nonnancols) - 1):
                if index == end: 
                    tab = table[col].dropna(how='all').drop_duplicates()
                    tab.columns=["names", "scores"]
                else: 
                    tab = tablecopy.iloc[:,[(len(tablecopy.columns)-3),
                                            (len(tablecopy.columns) -2),
                                            (len(tablecopy.columns) -1)] ].dropna(how='all')
                    tab.columns=["names1", 'names2', "scores"]
                    tab['scores'].iloc[0] = tab['names2'].iloc[0]
                    tab['names2'].iloc[0] = tab['names1'].iloc[0]
                    tab = tab.drop(columns='names1')
                    tab.columns =["names", "scores"]


            else:   
                tab = table[col].dropna(how='all').drop_duplicates()
                tab.columns=["names", "scores"]


            # format Scores
            tab['scores'] = [re.sub(r'\D+', '', str(x)) for x in tab.scores]
            tab['scores'].replace('', np.nan, inplace=True)
            tab = tab.dropna(subset = ['scores'], how='all')

            nrows = tab.shape[0]
    #         print(tab)

            if nrows== 2: 
                nrows = 1
            for pair in range(0, nrows, 2):

                pair1 = pair
                pair2 = pair + 1
                def name_cleaner(tab, index): 
                    name = str(tab['names'].iloc[index])
                    if name[len(name)-1] == ']': 
                        name = name[0:(len(name)-5)]
                    return(name)
                result = {}


                result["playerA"] = name_cleaner(tab, pair1)
                result["playerB"] = name_cleaner(tab, pair2)
                result["playerARacks"] = tab['scores'].iloc[pair1] 
                result["playerBRacks"] = tab['scores'].iloc[pair2]
                result["matchIndex"] = matchIndex
                matchIndex += 1
                results_lst.append(result)

    wpa2018 = pd.DataFrame(results_lst)
    return(wpa2018)



def wpa_scraper_full(url, date, comp, begin, end): 
    tables = pd.read_html(url)
    wpa = wpa_scraper(tables, begin, end)
    wpa['date'] = date
    wpa['competition'] = comp
    return(wpa)

def wpa_scraper_single_table(table):
    results_lst = []
    matchIndex = 1 

    # Table
    table = table.dropna(axis=1, how='all').dropna(axis=0, how='all')

    new_header = table.iloc[0] #grab the first row for the header
    table = table[1:] #take the data less the header row
    table.drop(table.tail(1).index,inplace=True) # drop last n rows
    table.columns = new_header #set the header row as the df header      
    tablecopy = table.copy()

    table = table.loc[:, table.columns.notnull()]
    nonnancols = list(table.columns.unique())
    for idx, col in enumerate(nonnancols):
        tab = table[col].dropna(how='all').drop_duplicates()
        tab.columns=["names", "scores"]


        # format Scores
        tab['scores'] = [re.sub(r'\D+', '', str(x)) for x in tab.scores]
        tab['scores'].replace('', np.nan, inplace=True)
        tab = tab.dropna(subset = ['scores'], how='all')

        nrows = tab.shape[0]
#         print(tab)

        if nrows== 2: 
            nrows = 1
        for pair in range(0, nrows, 2):

            pair1 = pair
            pair2 = pair + 1
            def name_cleaner(tab, index): 
                name = str(tab['names'].iloc[index])
                if name[len(name)-1] == ']': 
                    name = name[0:(len(name)-5)]
                if name[0:2].isupper():
                    name = name[4:len(name)]
                return(name)
            result = {}


            result["playerA"] = name_cleaner(tab, pair1)
            result["playerB"] = name_cleaner(tab, pair2)
            result["playerARacks"] = tab['scores'].iloc[pair1] 
            result["playerBRacks"] = tab['scores'].iloc[pair2]
            result["matchIndex"] = matchIndex
            matchIndex += 1
            results_lst.append(result)

    wpa = pd.DataFrame(results_lst)
    return(wpa)




In [146]:
# Scrape pages 
wpa2018 = wpa_scraper_full("https://en.wikipedia.org/wiki/2018_WPA_World_Nine-ball_Championship#Preliminary_round", datetime.datetime(2018, 12, 10),
                           "2018 WPA World 9-Ball Championship", 2, 18)
wpa2017 = wpa_scraper_full("https://en.wikipedia.org/wiki/2017_WPA_World_Nine-ball_Championship#Preliminary_round", datetime.datetime(2017, 12, 9),
                           "2017 WPA World 9-Ball Championship", 2, 17)
wpa2016 = wpa_scraper_full("https://en.wikipedia.org/wiki/2016_WPA_World_Nine-ball_Championship#Preliminary_round", datetime.datetime(2016, 6, 30),
                           "2016 WPA World 9-Ball Championship", 2, 18)
wpa2015 = wpa_scraper_full("https://en.wikipedia.org/wiki/2015_WPA_World_Nine-ball_Championship#Preliminary_round", datetime.datetime(2015, 9, 7),
                           "2015 WPA World 9-Ball Championship", 2, 18)


In [147]:
wpa15_18 = pd.merge(wpa2018, wpa2017, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')
wpa15_18 = pd.merge(wpa15_18, wpa2016, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')
wpa15_18 = pd.merge(wpa15_18, wpa2015, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')

wpa15_18 = wpa15_18.sort_values(by= ['date', 'matchIndex']).reset_index(drop=True)
wpa15_18

,playerA,playerB,playerARacks,playerBRacks,matchIndex,date,competition
0,Hunter Lombardo,Bashar Hussain,9,8,1,2015-09-07,2015 WPA World 9-Ball Championship
1,Irsal Nasution,Albin Ouschan,8,9,2,2015-09-07,2015 WPA World 9-Ball Championship
2,Khaled Farag,Hunter Lombardo,5,9,3,2015-09-07,2015 WPA World 9-Ball Championship
3,Irsal Nasution,Konstantin Stepanow,9,5,4,2015-09-07,2015 WPA World 9-Ball Championship
4,Albin Ouschan,Khaled Farag,9,5,5,2015-09-07,2015 WPA World 9-Ball Championship
...,...,...,...,...,...,...,...
876,Carlo Biado,Wu Kun-lin,11,4,219,2018-12-10,2018 WPA World 9-Ball Championship
877,Shane Van Boening,Chang Jung-lin,11,8,220,2018-12-10,2018 WPA World 9-Ball Championship
878,Joshua Filler,Alexander Kazakis,11,10,221,2018-12-10,2018 WPA World 9-Ball Championship
879,Carlo Biado,Shane Van Boening,11,6,222,2018-12-10,2018 WPA World 9-Ball Championship


In [148]:
wpa15_18.to_csv("data/WPA2015_18.csv")

In [149]:
wpa15_18

,playerA,playerB,playerARacks,playerBRacks,matchIndex,date,competition
0,Hunter Lombardo,Bashar Hussain,9,8,1,2015-09-07,2015 WPA World 9-Ball Championship
1,Irsal Nasution,Albin Ouschan,8,9,2,2015-09-07,2015 WPA World 9-Ball Championship
2,Khaled Farag,Hunter Lombardo,5,9,3,2015-09-07,2015 WPA World 9-Ball Championship
3,Irsal Nasution,Konstantin Stepanow,9,5,4,2015-09-07,2015 WPA World 9-Ball Championship
4,Albin Ouschan,Khaled Farag,9,5,5,2015-09-07,2015 WPA World 9-Ball Championship
...,...,...,...,...,...,...,...
876,Carlo Biado,Wu Kun-lin,11,4,219,2018-12-10,2018 WPA World 9-Ball Championship
877,Shane Van Boening,Chang Jung-lin,11,8,220,2018-12-10,2018 WPA World 9-Ball Championship
878,Joshua Filler,Alexander Kazakis,11,10,221,2018-12-10,2018 WPA World 9-Ball Championship
879,Carlo Biado,Shane Van Boening,11,6,222,2018-12-10,2018 WPA World 9-Ball Championship


In [168]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2014_WPA_World_Nine-ball_Championship#Preliminary_round")
wpa2014 = wpa_scraper_single_table(tables[11])
wpa2014['date'] = datetime.datetime(2014, 6, 21)
wpa2014['competition'] = "2014 WPA World 9-Ball Championship"
wpa2014.to_csv("data/WPA2014.csv")


In [176]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2013_WPA_World_Nine-ball_Championship#Preliminary_round")

wpa2013 = wpa_scraper_single_table(tables[12])
wpa2013['date'] = datetime.datetime(2013, 9, 2)
wpa2013['competition'] = "2013 WPA World 9-Ball Championship"
wpa2013.to_csv("data/WPA2013.csv")


,playerA,playerB,playerARacks,playerBRacks,matchIndex,date,competition
0,Chang Jung-Lin,Oliver Ortmann,6,11,1,2013-09-02,2013 WPA World 9-Ball Championship
1,Antonio Gabica,Ang Tuan Nguyen,11,8,2,2013-09-02,2013 WPA World 9-Ball Championship
2,Ko Pin-yi,Ryu Seung-woo,11,2,3,2013-09-02,2013 WPA World 9-Ball Championship
3,Mika Immonen,Alex Pagulayan,11,6,4,2013-09-02,2013 WPA World 9-Ball Championship
4,Wu Jiaqing,Chao Fong-pang,11,8,5,2013-09-02,2013 WPA World 9-Ball Championship
...,...,...,...,...,...,...,...
58,Jeffrey de Luna,Thorsten Hohmann,7,11,59,2013-09-02,2013 WPA World 9-Ball Championship
59,Chris Melling,Carlo Biado,3,11,60,2013-09-02,2013 WPA World 9-Ball Championship
60,Antonio Gabica,Karl Boyes,11,6,61,2013-09-02,2013 WPA World 9-Ball Championship
61,Thorsten Hohmann,Carlo Biado,11,4,62,2013-09-02,2013 WPA World 9-Ball Championship


In [180]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2012_WPA_World_Nine-ball_Championship#Preliminary_round")

wpa2012 = wpa_scraper_single_table(tables[2])
wpa2012['date'] = datetime.datetime(2012, 6, 22)
wpa2012['competition'] = "2012 WPA World 9-Ball Championship"
wpa2012.to_csv("data/WPA2012.csv")
wpa2012

,playerA,playerB,playerARacks,playerBRacks,matchIndex,date,competition
0,Ralf Souquet,Nick Malai,11,4,1,2012-06-22,2012 WPA World 9-Ball Championship
1,Jonny Martinez,Takashi Uraoka,11,8,2,2012-06-22,2012 WPA World 9-Ball Championship
2,Fu Che-wei,Andrea Klasovic,11,4,3,2012-06-22,2012 WPA World 9-Ball Championship
3,Jayson Shaw,Bruno Muratore,6,11,4,2012-06-22,2012 WPA World 9-Ball Championship
4,Aki Heiskanen,Hsu Kai-lun,5,11,5,2012-06-22,2012 WPA World 9-Ball Championship
...,...,...,...,...,...,...,...
58,Naoyuki Ōi,Karl Boyes,11,9,59,2012-06-22,2012 WPA World 9-Ball Championship
59,Darren Appleton,Ko Pin-yi,11,9,60,2012-06-22,2012 WPA World 9-Ball Championship
60,Ralf Souquet,Li Hewen,6,11,61,2012-06-22,2012 WPA World 9-Ball Championship
61,Naoyuki Ōi,Darren Appleton,7,11,62,2012-06-22,2012 WPA World 9-Ball Championship


In [195]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2011_WPA_World_Nine-ball_Championship#Preliminary_round")
# 1 to 9
tables[4]

def wpa_scraper_2011(tables, begin, end):
    results_lst = []
    matchIndex = 1 

    # By inspection, choose interesting tables from beginning to end.
    for index in range(begin, end + 1, 1):
        table = tables[index]

        # Table
        table = table.dropna(axis=1, how='all').dropna(axis=0, how='all')

        new_header = table.iloc[0] #grab the first row for the header
        table = table[1:] #take the data less the header row
        table.drop(table.tail(1).index,inplace=True) # drop last n rows
        table.columns = new_header #set the header row as the df header      
        tablecopy = table.copy()

        table = table.loc[:, table.columns.notnull()]
        nonnancols = list(table.columns.unique())
        for idx, col in enumerate(nonnancols):

            tab = table[col].dropna(how='all').drop_duplicates()

            tab.columns=["namesextra","names", "scores"]


            # format Scores
            tab['scores'] = [re.sub(r'\D+', '', str(x)) for x in tab.scores]
            tab['scores'].replace('', np.nan, inplace=True)
            tab = tab.dropna(subset = ['scores'], how='all')

            nrows = tab.shape[0]
    #         print(tab)

            if nrows== 2: 
                nrows = 1
            for pair in range(0, nrows, 2):

                pair1 = pair
                pair2 = pair + 1
                def name_cleaner(tab, index): 
                    name = str(tab['names'].iloc[index])
                    if name[len(name)-1] == ']': 
                        name = name[0:(len(name)-5)]
                    return(name)
                result = {}


                result["playerA"] = name_cleaner(tab, pair1)
                result["playerB"] = name_cleaner(tab, pair2)
                result["playerARacks"] = tab['scores'].iloc[pair1] 
                result["playerBRacks"] = tab['scores'].iloc[pair2]
                result["matchIndex"] = matchIndex
                matchIndex += 1
                results_lst.append(result)

    wpa2018 = pd.DataFrame(results_lst)
    return(wpa2018)


wpa2011 = wpa_scraper_2011(tables, 1 , 9 )
wpa2011['date'] = datetime.datetime(2011, 6, 25)
wpa2011['competition'] = "2011 WPA World 9-Ball Championship"
wpa2011.to_csv("data/WPA2011.csv")
wpa2011

,playerA,playerB,playerARacks,playerBRacks,matchIndex,date,competition
0,Ralf Souquet,Serge Das,11,5,1,2011-06-25,2011 WPA World 9-Ball Championship
1,Radosław Babica,Mariusz Skoneczny,7,11,2,2011-06-25,2011 WPA World 9-Ball Championship
2,Thorsten Hohmann,Lo Li-wen,7,11,3,2011-06-25,2011 WPA World 9-Ball Championship
3,Nick van den Berg,Vicenancio Tanio,7,11,4,2011-06-25,2011 WPA World 9-Ball Championship
4,Ralf Souquet,Mariusz Skoneczny,11,2,5,2011-06-25,2011 WPA World 9-Ball Championship
...,...,...,...,...,...,...,...
58,Ronnie Alcano,Tōru Kuribayashi,11,10,59,2011-06-25,2011 WPA World 9-Ball Championship
59,Dennis Orcullo,Daryl Peach,11,4,60,2011-06-25,2011 WPA World 9-Ball Championship
60,Yukio Akakariyama,Mark Gray,11,10,61,2011-06-25,2011 WPA World 9-Ball Championship
61,Ronnie Alcano,Dennis Orcullo,11,9,62,2011-06-25,2011 WPA World 9-Ball Championship


In [220]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2010_WPA_World_Nine-ball_Championship#Preliminary_round")
wpa2010 = wpa_scraper(tables, 2, 21)
lastIndex = max(wpa2010.matchIndex)
tab = wpa_scraper_2011(tables, 22, 22)
tab['matchIndex'] = tab['matchIndex'] + lastIndex
wpa2010
wpa2010 = pd.merge(wpa2010, tab, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'matchIndex'], 
                  how='outer')
wpa2010['date'] = datetime.datetime(2010, 6, 29)
wpa2010['competition'] = "2010 WPA World 9-Ball Championship"
wpa2010.to_csv("data/WPA2010.csv")


In [221]:
wpafull = pd.merge(wpa15_18, wpa2014, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')
wpafull = pd.merge(wpafull, wpa2013, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')
wpafull = pd.merge(wpafull, wpa2012, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')
wpafull = pd.merge(wpafull, wpa2011, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')
wpafull = pd.merge(wpafull, wpa2010, 
                    on=["playerA", "playerB", "playerARacks",
                        "playerBRacks", 'date', 'matchIndex',
                        "competition"], 
                  how='outer')

wpafull = wpafull.sort_values(by= ['date', 'matchIndex']).reset_index(drop=True)
wpafull

,playerA,playerB,playerARacks,playerBRacks,matchIndex,date,competition
0,Mazen Berjawi,Mariusz Skoneczny,9,2,1,2010-06-29,2010 WPA World 9-Ball Championship
1,Hasan Zeraatgar,Bruno Muratore,3,9,2,2010-06-29,2010 WPA World 9-Ball Championship
2,Abdulla a. Al Daham,Mazen Berjawi,6,9,3,2010-06-29,2010 WPA World 9-Ball Championship
3,Imran Majid,Bruno Muratore,7,9,4,2010-06-29,2010 WPA World 9-Ball Championship
4,Shane Van Boening,Abdulla a. Al Daham,9,6,5,2010-06-29,2010 WPA World 9-Ball Championship
...,...,...,...,...,...,...,...
1354,Carlo Biado,Wu Kun-lin,11,4,219,2018-12-10,2018 WPA World 9-Ball Championship
1355,Shane Van Boening,Chang Jung-lin,11,8,220,2018-12-10,2018 WPA World 9-Ball Championship
1356,Joshua Filler,Alexander Kazakis,11,10,221,2018-12-10,2018 WPA World 9-Ball Championship
1357,Carlo Biado,Shane Van Boening,11,6,222,2018-12-10,2018 WPA World 9-Ball Championship


In [222]:
wpafull.to_csv("data/WPAFull.csv")

# World Pool Masters

In [226]:
def wpm_scraper(tables, begin, end):
    results_lst = []
    matchIndex = 1 

    # By inspection, choose interesting tables from beginning to end.
    for index in range(begin, end + 1, 1):
        table = tables[index]

        # Table
        table = table.dropna(axis=1, how='all').dropna(axis=0, how='all')

        new_header = table.iloc[0] #grab the first row for the header
        table = table[1:] #take the data less the header row
        table.drop(table.tail(1).index,inplace=True) # drop last n rows
        table.columns = new_header #set the header row as the df header      
        tablecopy = table.copy()

        table = table.loc[:, table.columns.notnull()]
        nonnancols = list(table.columns.unique())
        for idx, col in enumerate(nonnancols):

            tab = table[col].dropna(how='all').drop_duplicates()

            tab.columns=["rankings","names", "scores"]


            # format Scores
            tab['scores'] = [re.sub(r'\D+', '', str(x)) for x in tab.scores]
            tab['scores'].replace('', np.nan, inplace=True)
            tab = tab.dropna(subset = ['scores'], how='all')

            nrows = tab.shape[0]
    #         print(tab)

            if nrows== 2: 
                nrows = 1
            for pair in range(0, nrows, 2):

                pair1 = pair
                pair2 = pair + 1
                def name_cleaner(tab, index): 
                    name = str(tab['names'].iloc[index])
                    if name[len(name)-1] == ']': 
                        name = name[0:(len(name)-5)]
                    return(name)
                result = {}


                result["playerA"] = name_cleaner(tab, pair1)
                result["playerB"] = name_cleaner(tab, pair2)
                result["playerARacks"] = tab['scores'].iloc[pair1] 
                result["playerBRacks"] = tab['scores'].iloc[pair2]
                result["matchIndex"] = matchIndex
                matchIndex += 1
                results_lst.append(result)

    wpm = pd.DataFrame(results_lst)
    return(wpm)

In [231]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2019_World_Pool_Masters#Main_draw")
tables[2]
# wpm2019 = wpm_scraper(tables, 2, 2)

# wpm2019
# wpa2010 = pd.merge(wpa2010, tab, 
#                     on=["playerA", "playerB", "playerARacks",
#                         "playerBRacks", 'matchIndex'], 
# #                   how='outer')
# wpa2010['date'] = datetime.datetime(2010, 6, 29)
# wpa2010['competition'] = "2010 WPA World 9-Ball Championship"
# wpa2010.to_csv("data/WPA2010.csv")


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,Preliminary Round First to 7 racks,Preliminary Round First to 7 racks,Preliminary Round First to 7 racks,NaN,NaN,Last 16 First to 7 racks,Last 16 First to 7 racks,Last 16 First to 7 racks,NaN,...,Semifinals First to 8 racks,Semifinals First to 8 racks,Semifinals First to 8 racks,NaN,NaN,Final First to 9 racks,Final First to 9 racks,Final First to 9 racks,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,11,David Alcaide (ESP),7,NaN,NaN,1,Niels Feijen (NED),3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,12,Francisco Sánchez Ruíz (ESP),3,NaN,NaN,11,David Alcaide (ESP),7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,12,Francisco Sánchez Ruíz (ESP),3,NaN,NaN,11,David Alcaide (ESP),7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,10,Shane Van Boening (USA),7,NaN,NaN,8,Han Yu (CHN),6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,10,Shane Van Boening (USA),7,NaN,NaN,8,Han Yu (CHN),6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,19,Kelly Fisher (ENG),2,NaN,NaN,10,Shane Van Boening (USA),7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Knitting Things Together

### Accustats

In [13]:
accustats = pd.read_csv("data/accustats.csv")
# accustats['Date'] = pd.to_datetime(accustats.matchTime)

accustats = accustats.sort_values(by = "matchIndex")
accustats = accustats.rename(columns = {'matchTime':'date', 
                                       "Competition": 'competition'})
accustats = accustats.drop(columns = ['round'])
accustats.date = pd.to_datetime(accustats['date'])


### Eurotour

In [14]:
eurotour = pd.read_csv("data/EuroTourTreviso2020.csv")


In [15]:
# Clean names
import regex as re

def clean_names(df, column):
    result = []
    for idx, row in df.iterrows(): 
        names = re.split('[\\s]+', row[column])
        lastname = ""
        firstname = ""
        for name in names: 
            if name.isupper():
                lastname += str.title(name)
            elif name == "Jr.":
                pass
            else: 
                firstname += str.title(name) + " "
        fullname = firstname + lastname
        result.append(fullname)
    return(result)


In [16]:
eurotour.playerA = clean_names(eurotour, "playerA")
eurotour.playerB = clean_names(eurotour, "playerB")
eurotour = eurotour.rename(columns={"matchTime":'date'})
eurotour = eurotour.drop(columns = ['round'])
eurotour['competition'] = "2020 EuroTour Treviso"
eurotour.date = pd.to_datetime(eurotour['date'])

### WPA 2019

In [17]:
wpa = pd.read_csv("data/WPAW9Ball.csv")
wpa = wpa.rename(columns = {"WinnerName":"playerA", 
                      "LoserName": "playerB", 
                      "WinnerRacks":"playerARacks", 
                      "LoserRacks":"playerBRacks",
                      "Round":"round", 
                      "Date":'date',
                      "GameID": 'matchIndex'})
wpa['competition'] = "WPA World 9-Ball Championship"
wpa = wpa.drop(columns = 'round')
wpa.date = pd.to_datetime(wpa['date'])

### WPA 2018-2010

In [18]:
wpa2 = pd.read_csv('data/WPAFull.csv')
wpa2.date = pd.to_datetime(wpa2['date'])

### US Open

In [19]:
usopen = pd.read_csv("data/usopen2019.csv")
usopen = usopen.rename(columns = {"winner":"playerA", 
                      "loser": "playerB", 
                      "winnerScore":"playerARacks", 
                      "loserScore":"playerBRacks",
                      "Round":"round", 
                      "matchTime":'date',
                      "matchNum": 'matchIndex', 
                      'tournament':'competition'})
usopen = usopen.drop(columns = ["winnerNationality", "winnerRunouts", 
                                "loserNationality", "loserRunouts","Unnamed: 0" ])
usopen.date = pd.to_datetime(usopen['date'])

### Turning Stone Classic 2017

In [20]:
tstone = pd.read_csv("data/turningstone2017.csv")
tstone = tstone.rename(columns = {"winner":"playerA", 
                      "loser": "playerB", 
                      "winnerScore":"playerARacks", 
                      "loserScore":"playerBRacks",
                      "Round":"round", 
                      "matchTime":'date',
                      "matchNum": 'matchIndex', 
                      'tournament':'competition'})
tstone = tstone.drop(columns = ["winnerNationality", "winnerRunouts", 
                                "loserNationality", "loserRunouts","Unnamed: 0" ])
tstone.date = pd.to_datetime(tstone['date'])
tstone['competition'] = "2017 Turning Stone Classic XXVII 9-Ball Open"

### World Pool Masters 2010-2019

In [21]:
wpm = pd.read_csv('data/worldpoolmasters.csv')
wpm.date = pd.to_datetime(wpm['date'])


### Merge

In [29]:
fulldf1 = pd.merge(accustats, eurotour, on=["playerA", "playerB", "playerARacks",
                                           "playerBRacks", 'date', 'matchNumber',
                                          "competition"], 
                  how='outer')
fulldf2 = pd.merge(fulldf1, wpa, on=["playerA", "playerB", "playerARacks",
                                   "playerBRacks", 'date', 'matchIndex',
                                          "competition"], 
                  how='outer')
fulldf3 = pd.merge(fulldf2, usopen, on=["playerA", "playerB", "playerARacks",
                                           "playerBRacks", 'date', 'matchIndex',
                                          "competition"], 
                  how='outer')
fulldf4 = pd.merge(fulldf3, tstone, on=["playerA", "playerB", "playerARacks",
                                           "playerBRacks", 'date', 'matchIndex',
                                          "competition"], 
                  how='outer')

fulldf5 = pd.merge(fulldf4, wpa2, on=["playerA", "playerB", "playerARacks",
                                   "playerBRacks", 'date', 'matchIndex',
                                          "competition"], 
                  how='outer')

fulldf6 = pd.merge(fulldf5, wpm, on=["playerA", "playerB", "playerARacks",
                                   "playerBRacks", 'date', 'matchIndex',
                                          "competition"], 
                  how='outer')
fulldf6['playerA'] = fulldf6['playerA'].str.strip()
fulldf6['playerB'] = fulldf6['playerB'].str.strip()
fulldf = fulldf6
fulldf.head()

,matchIndex,matchNumber,date,playerA,playerARacks,playerB,playerBRacks,competition,Unnamed: 0
0,1.0,329B-01,2007-10-14,Shane Van Boening,11,Ronnie Alcano,4,2007 US Open 9-Ball,NaN
1,2.0,329B-02,2007-10-14,Ernesto Dominguez,11,Frankie Hernandez,5,2007 US Open 9-Ball,NaN
2,3.0,329B-03,2007-10-14,Tony Robles,11,Rafael Martinez,5,2007 US Open 9-Ball,NaN
3,4.0,329B-04,2007-10-14,Louis Ulrich,11,Alex Pagulayan,7,2007 US Open 9-Ball,NaN
4,5.0,329B-05,2007-10-14,Francisco Bustamante,11,Richie Orem,4,2007 US Open 9-Ball,NaN


In [31]:
def deduplicate_names(series):
    def switcher(argument):
        switcher = {
            'Abderrahmane Mebarki': 'Abder Rehman Mebarki', 
            'Ahmed M. Salah': 'Ahmad Mohamad Sllah',
            '(Kevin) Cheng Yu Hsuan': 'Kevin Cheng',
            'Cheng Yu-hsuan': 'Kevin Cheng',
            'YUKIO AKAGARIYAMA':'Yukio Akakariyama', 
            'Yukio Akagariyama':'Yukio Akakariyama', 
            'Akagariyama Yukio':'Yukio Akakariyama', 
            'Abdulla al-Yousif':  'Abdulla al-Yousif',
            'Alexander Kazakis': 'Alex Kazakis',
            'Ali Abdulhadi':  'Ali Abdulhadi Almeri',
            'Andrew Kong Bu Hong' :'Andrew Kong',
            'Artem Koshovoj':'Artem Koshovyi',
            'Badr al-Awadi':'Bader al-Awadhi',
            'Chang Jun Lin': 'Chang Jung-Lin',
            'Chang Jun Lin': 'Chang Jung-Lin',
            'Chang Jung-Lin': 'Chang Jung-Lin',
            'Chang Yu-lung': 'Chang Yu-Lung',
            'Chang Yu Lung': 'Chang Yu-Lung',
            'Chao Fong-pang':'Chao Fong-Pang',
            'Charles Williams':'Charlie Williams',
            'Dang Jinhu':'Dang Jin Hu',
            'David Alcaide Bermudez': 'David Alcaide',
            'Dennis Orcullo':'Dennis Orcollo',
            'Donnie Mills':'Donny Mills',
            'Dương Quốc Hoàng': 'Duong Quac Hoang',
            'Eklent Kaçi':'Eklent Kaci',
            'Eklent KaÇi': 'Eklent Kaci',
            'Erik  Hjorleifson':'Erik Hjorleifson',
            'Fjodor Gorst':'Fedor Gorst',
            'Francisco Díaz-Pizarro':'Francisco Diaz-Pizarro',
            'Francisco Sánchez Ruíz':'Francisco Sanchez Ruiz',
            'Francisco Sanchez-Ruiz':'Francisco Sanchez Ruiz',
            'Francisco Sánchez': 'Francisco Sanchez Ruiz',
            'Franklin Hernandez':'Frankie Hernandez',
            'HAYATO HIJIKATA':'Hayato Hijikata',
            'Hsieh Chia-chen': 'Hsieh Chia-Chen',
            'Hsu Kai-lun':'Hsu Kai Lun',
            'Jalal al-Sarisi':'Jalal Al Sarisi',
            'Jeff de Luna':'Jeffrey de Luna',
            'Jeffrey De Luna': 'Jeffrey de Luna',
            'Jeong Young-hwa':'Jeong Young Hwa',
            'Jeremy jones':'Jeremy Jones',
            'Jung Lin Chang':'Jung-Lin Chang',
            'Kai Lun Hsu':'Kai-Lun Hsu',
            'Ko Pin-Yi (秉逸柯)': 'Ko Pin Yi',
            'Ko Pin-yi':'Ko Pin Yi',
            'Ko Ping-chung': 'Ko Ping-Chung',
            'Ko Ping Chung': 'Ko Ping-Chung',
            'Ko Ping-han': 'Ko Ping Han',
            'konomopoulos': 'Nikos Ekonomopoulos',
            'Konrad Juszczyszn': 'Konrad Juszczyszyn',
            'Konstantin Stepanow':'Konstantin Stepanov',
            'Konstantinos Koukiadakis':'Kostas Koukiadakis',
            'Kuo Po Chen':'Kuo Po-Cheng',
            'Lee jun-ho':'Lee Jun-ho',
            'Lee Van Corteza': 'Lee Vann Corteza',
            'Li Hewen':'Li He Wen',
            'Liu Cheng-chieh': 'Liu Cheng Chieh',
            'Liu Haitao (pool player)': 'Liu Haitao',
            'Liu Hai Tao': 'Liu Haitao',
            'Liu Ri-teng':'Liu Ri Teng',
            'Lo Li-wen':'Lo Li-Wen',
            'Lưu Minh Phúc':'Luu Minh Phuc',
            'Majed Alazmi':'Majid N. Alazmi', 
            'Marc Vidal Claramunt': 'Marc Vidal',
            'MASATO YOSHIOKA':'Masato Yoshioka',
            'Mateusz Śniegocki':'Mateusz Sniegocki',
            'Matthew Edwards': 'Matt Edwards',
            'Maxim Dudanez':'Maxim Dudanets',
            'Maximillian Lechner':'Maximilian Lechner',
            'Mazen Berjaoui':'Mazen Berjawi',
            'Mazen Berjuai':'Mazen Berjawi',
            'Mazin Berjawi':'Mazen Berjawi',
            'Md al-Amin':'MD Almin',
            'Meshal al-Murdhi': 'Meshaal al-Murdhi',
            'Mieszko Fortuński':'Mieszko Fortunski',
            'Miesko Fortunski': 'Mieszko Fortunski',
            'Mohammed Berjaui':  'Mohammad Berjawi',
            'Mohammad Berjaoui':  'Mohammad Berjawi',
            'Mohammad Sharif (pool player)':'Mohammad Sharif',
            'Mohammed Saeed (pool player)':'Mohammed Saeed',
            'Muhammad Zulfikiri': 'Muhammad Zulfikri',
            'Naoyuki Ōi': 'Naoyuki Oi',
            'Omar Al-Shaheen':'Omar al-Shaheen',
            'Omar Al Shaheen': 'Omar al-Shaheen',
            'Óscar Domínguez':'Oscar Dominguez',
            'Radislaw Babica': 'Radoslaw Babica',
            'Ralf Souqet':'Ralf Souquet',
            'Raymund Faron':'Raymund Faraon',
            'Raymund Farun':'Raymund Faraon',
            'Ronato Alcano': 'Ronnie Alcano',
            'Ruslan Chinachov': 'Ruslan Chinakhov',
            'Ruslan Chinahov': 'Ruslan Chinakhov',
            'Ruslan Tschinachow': 'Ruslan Chinakhov',
            'Salah al-Remawi':'Salah Al Riwani',
            'Salaheldeen Al Rimawi':'Salah Al Riwani',
            'Sanjin Pehlivanović': 'Sanjin Pehlivanovic',
            'Sergei Luzker':  'Sergey Lutsker',
            'Stanimir Ruslanov':  'Stanimir Ruslanow',
            'Tomasz Kapłan':'Tomasz Kaplan',
            'Tōru Kuribayashi':'Toru Kurabayashi',
            'Wang Can (pool player)': 'Wang Can',
            'Wiktor Zieliński':'Wiktor Zielinski',
            'Wojciech Sroczyński':'Wojciech Sroczynski',
            'Wu Jia-qing': 'Wu Jiaqing',
            'Wu Kun Lin': 'Wu Kun-Lin',
            'Wu Kun-lin': 'Wu Kun-Lin',
            'Yang Ching-shun': 'Yang Ching Shun',
            'Yip Kin Ling Leo': 'Yip Kin Ling',
            'Yousseff Jalar':'Yousseff Jaffar',
            'Zhou Long (Poolbillardspieler)\xa0[Zhou ': 'Zhou Long',            
        }

        # get() method of dictionary data type returns 
        # value of passed argument if it is present 
        # in dictionary otherwise second argument will
        # be assigned as default value of passed argument
        val = switcher.get(argument, "nothing")
        if val == "nothing":
            return argument
        else:
#             print(val)
            return val
    
    lst = []
    for idx, item in enumerate(series): 
        lst.append(switcher(item))
#     print(lst)
    return(lst)
   
    

fulldf["playerA"] = deduplicate_names(fulldf["playerA"])

print("Should be 1: " + str(np.sum(fulldf["playerB"]== "Abder Rehman Mebarki")))
fulldf["playerB"] = deduplicate_names(fulldf["playerB"])

print("Should be 2: " + str(np.sum(fulldf["playerB"]== "Abder Rehman Mebarki")))


Should be 1: 2
Should be 2: 2


In [32]:
fulldf

,matchIndex,matchNumber,date,playerA,playerARacks,playerB,playerBRacks,competition,Unnamed: 0
0,1.0,329B-01,2007-10-14,Shane Van Boening,11,Ronnie Alcano,4,2007 US Open 9-Ball,NaN
1,2.0,329B-02,2007-10-14,Ernesto Dominguez,11,Frankie Hernandez,5,2007 US Open 9-Ball,NaN
2,3.0,329B-03,2007-10-14,Tony Robles,11,Rafael Martinez,5,2007 US Open 9-Ball,NaN
3,4.0,329B-04,2007-10-14,Louis Ulrich,11,Alex Pagulayan,7,2007 US Open 9-Ball,NaN
4,5.0,329B-05,2007-10-14,Francisco Bustamante,11,Richie Orem,4,2007 US Open 9-Ball,NaN
...,...,...,...,...,...,...,...,...,...
3097,27.0,NaN,2010-05-12,Toru Kuribayashi,8,Charlie Williams,6,2010 World Pool Masters,NaN
3098,28.0,NaN,2010-05-12,Thomas Engert,6,Oliver Ortmann,8,2010 World Pool Masters,NaN
3099,29.0,NaN,2010-05-12,Dennis Orcollo,8,Roberto Gomez,6,2010 World Pool Masters,NaN
3100,30.0,NaN,2010-05-12,Toru Kuribayashi,8,Oliver Ortmann,3,2010 World Pool Masters,NaN


In [33]:
fulldf.to_csv("data/fulldf.csv")

# Clean Test Dataset

In [35]:
import datetime
def date_switcher(argument):
    date_switcher_dict = {
        1: datetime.datetime(2021, 3,22),
        2: datetime.datetime(2021, 3,23),
        3: datetime.datetime(2021, 3,24),
        4: datetime.datetime(2021, 3,25),
        5: datetime.datetime(2021, 3,26),
        6: datetime.datetime(2021, 3,27),
        7: datetime.datetime(2021, 3,28),
        8: datetime.datetime(2021, 3,29)
    }

    # get() method of dictionary data type returns 
    # value of passed argument if it is present 
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    val = date_switcher_dict.get(argument, None)
    return(val)
    

data = pd.read_csv('data/predatorCLP.csv')
data["playerARacks"] = 5


dates = []
for _, row in data.iterrows():
#     print(row["Group"])
    dates.append(date_switcher(row.loc["Group"]))
# print(dates)
data["date"] = dates
data["competition"] = "2021 Predator CLP"
data = data.rename(columns={"GameID": "matchIndex", 
                     "WinnerName": "playerA", 
                     "LoserName": "playerB",
                    "LoserRacks": "playerBRacks"})
data = data.drop(columns = ["isFinal","isSemi", "Group", "Table"])

data = data[["matchIndex", "playerA", "playerB", "playerARacks", "playerBRacks", "playerARacks", "date", "competition"]]


data.to_csv("data/predatorCLPClean.csv")